## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from world_weather_analysis.config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Beloha,-25.17,45.05,64.60,70,0,7.78,MG,2020-08-24 02:28:48
1,1,Walvis Bay,-22.96,14.51,54.00,93,0,6.93,NaN,2020-08-24 02:28:49
2,2,Abha,18.22,42.51,71.60,56,59,4.18,SA,2020-08-24 02:28:49
3,3,Jamestown,42.10,-79.24,69.01,93,1,4.70,US,2020-08-24 02:28:49
4,4,Traverse City,44.76,-85.62,73.40,88,40,2.66,US,2020-08-24 02:27:04


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,Bathsheba,13.22,-59.52,80.60,83,40,8.05,BB,2020-08-24 02:28:52
22,22,Hambantota,6.12,81.12,80.60,94,20,5.82,LK,2020-08-24 02:28:54
27,27,Saint Anthony,45.02,-93.22,81.00,69,1,3.65,US,2020-08-24 02:27:07
28,28,Marsh Harbour,26.54,-77.06,85.24,76,1,24.83,BS,2020-08-24 02:28:55
36,36,Pedernales,18.04,-71.74,81.73,70,95,7.99,DO,2020-08-24 02:28:57
37,37,Ponta Do Sol,32.67,-17.10,75.20,64,20,20.80,PT,2020-08-24 02:28:57
39,39,Progreso,21.28,-89.67,75.99,94,90,4.70,MX,2020-08-24 02:28:57
44,44,Cockburn Town,21.46,-71.14,84.11,83,100,24.14,TC,2020-08-24 02:24:24
45,45,Bandarbeyla,9.49,50.81,75.36,71,89,29.35,SO,2020-08-24 02:28:59
49,49,Los Llanos De Aridane,28.66,-17.92,79.00,78,1,16.11,ES,2020-08-24 02:29:00


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0    0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       0
Date          0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#There were zero empty rows according to 4a.

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
13,Bathsheba,BB,80.60,13.22,-59.52,
22,Hambantota,LK,80.60,6.12,81.12,
27,Saint Anthony,US,81.00,45.02,-93.22,
28,Marsh Harbour,BS,85.24,26.54,-77.06,
36,Pedernales,DO,81.73,18.04,-71.74,
37,Ponta Do Sol,PT,75.20,32.67,-17.10,
39,Progreso,MX,75.99,21.28,-89.67,
44,Cockburn Town,TC,84.11,21.46,-71.14,
45,Bandarbeyla,SO,75.36,9.49,50.81,
49,Los Llanos De Aridane,ES,79.00,28.66,-17.92,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna()

,City,Country,Max Temp,Lat,Lng,Hotel Name
13,Bathsheba,BB,80.60,13.22,-59.52,Atlantis Hotel
22,Hambantota,LK,80.60,6.12,81.12,Bungalow 63
27,Saint Anthony,US,81.00,45.02,-93.22,Holiday Inn Express Roseville-St. Paul
28,Marsh Harbour,BS,85.24,26.54,-77.06,Abaco Beach Resort
36,Pedernales,DO,81.73,18.04,-71.74,Hotel Costa Marina
...,...,...,...,...,...,...
661,Yanam,IN,82.40,16.73,82.22,The Regency Hotel
667,Port Antonio,JM,80.01,18.18,-76.46,Trident Hotel
672,Tallahassee,US,75.20,30.44,-84.28,Four Points by Sheraton Tallahassee Downtown
677,Korla,CN,85.73,41.76,86.15,Gulisitan Hotel


In [12]:
# 8a. Create the output File (CSV)
hotel_df.to_csv('WeatherPy_Vacation.csv')
# 8b. Export the City_Data into a csv
hotel_df.to_csv('WeatherPy_Vacation.csv', index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
max_temp = hotel_df["Max Temp"]
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
hotel_layer = gmaps.marker_layer(locations,
                                 info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))